# Hadar Sharon - The Battle of the Neighborhoods

### The Business Problem:

I am looking to open a high-end Asian restaurant in Tel-Aviv, Israel. I am unsure as of where I should open it, as there are a couple of specific issues that bother me:
1. I want my restaurant to stand out from the crowd, meaning that if there are many Asian restaurants in my area, it would negatively affect my business traffic and cause customer loss with fierce competition.
2. On the other hand, if my restaurant is located in a quiet and/or neglected area, or rather a very noisy and industrial area, it would not attract any customers either.
3. Moreover - Since my restaurant is going to be a high-end restaurant, I need it to be close to an affluent neighbourhood(s) so as to attract affluent customers.
4. Accessibility is still important - I'd still like my restaurant to be relatively accessible, thus not located at the edge of the city or far from main roads.

### The Data:

I am going to need data based on the city of Tel-Aviv, Israel, as well as its neighbourhoods.

Specifically, I need to be able to map out all Asian restaurants in Tel-Aviv, and be able to obtain insights about the residents of each neighbourhood - their income, their age, and lifestyle in general (as possible).

I am planning on using the Foursquare API to handle all the Geolocation data related to restaurants, and perhaps bound & classify each neighbourhood of the city using it (+ Visualization).

I am also looking to obtain a dataset with data about the residents of Tel-Aviv, whether from a local API or I will scrape entire datasets from the Internet if needed.

### Methodology

First, we are going to discuss the libraries and modules used in this report.
As I chose to use Python as my main programming language for this task, I have relied on its popular Data Analysis, namely:
- Numpy, Pandas for Exploratory Data Analysis
- Matplotlib, Folium for Data Visualization
- I also used the Foursquare API for Geolocation data

###### Let's import these libraries: 

In [1]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import folium
# Foursquare credentials
FS_CLIENT_ID = "HWMWGYDHIK1T1RY5AGIRVO5DSBTNK4BYZJZY12DBXIHZEVHN"
FS_CLIENT_SECRET = "KVNRQT1IYIQK332KWN2VH1BSUCRHJQMHBFRFH200AIR2CBHM"
FS_VERSION = "20200306"

###### And create a folium map object focused on Tel-Aviv:

In [2]:
from IPython.display import display
telaviv_lat = 32.0853
telaviv_lon = 34.7818
telaviv_map = folium.Map(location=[telaviv_lat, telaviv_lon], zoom_start=13)
telaviv_map

#### Now, let's make some requests to Foursquare in order to map all Asian restaurants in Tel Aviv

First, we're going to get all Foursquare categories, so we could filter our results to match only Asian restaurants
(Note: this includes many subcategories i.e. Japanese, Chinese, Dim-Sum places etc., so it is quite tedious!)

In [3]:
import json, requests

# Find the relevant Foursquare category for Asian restaurants so we could query by it later
fs_categories_url = "https://api.foursquare.com/v2/venues/categories"
fs_categories_params = dict(
    client_id=f'{FS_CLIENT_ID}',
    client_secret=f'{FS_CLIENT_SECRET}',
    v=f'{FS_VERSION}'
)

resp = requests.get(url=fs_categories_url, params=fs_categories_params)
data = json.loads(resp.text)

###### Out of all categories found, we want to keep a list of only the Asian categories and the aforementioned subtypes
(This is done here with pandas dataframes and a recursive function that adds all relevant category ids)

In [4]:
categories_to_search_by = []

def add_category_id(entry, result_list):
    result_list.append(entry['id'])
    if 'categories' in entry:
        for sub_entry in entry['categories']:
            add_category_id(sub_entry, result_list)

categories_df = pd.DataFrame(data['response']['categories'])
food_categories_df = categories_df[categories_df['name'].isin(["Food", "Restaurant", "Restaurants"])]
asian_categories = list(filter(lambda res: res['shortName'] in 'Asian', food_categories_df['categories'].values[0]))

for category in asian_categories:
    add_category_id(category, categories_to_search_by)

#### Great! Now we have all the category IDs we want to filter by
On the one hand we filtered out non-Asian restaurants, but on the other hand we have all the Asian subcategories so we can be quite sure that we don't miss that Thai place around the corner! ;)

###### Now it's time to search for the actual restaurants, also using the Foursquare API:

In [5]:
fs_search_url = "https://api.foursquare.com/v2/venues/search"

fs_search_params = dict(
  client_id=f'{FS_CLIENT_ID}',
  client_secret=f'{FS_CLIENT_SECRET}',
  v=f'{FS_VERSION}',
  ll=f'{telaviv_lat}, {telaviv_lon}',
  categoryId=f"{','.join(categories_to_search_by)}",
  radius=10000  # 10 kilometers radius from the heart of Tel-Aviv
)
resp = requests.get(url=fs_search_url, params=fs_search_params)
found_restaurants_data = json.loads(resp.text)

###### Let's put all found restaurants into a nice pandas dataframe:

In [6]:
found_restaurants_df = pd.DataFrame(found_restaurants_data['response']['venues'])
found_restaurants_df.head()

,id,name,location,categories,referralId,hasPerk,venuePage
0,594800de4382ab30b243a4e6,Cà Phê Hanoi,"{'lat': 32.07968605952711, 'lng': 34.779864215...","[{'id': '4bf58dd8d48988d14a941735', 'name': 'V...",v-1583505798,False,NaN
1,572adbf5498e1d76d32da113,מלי מלי,"{'lat': 32.081486, 'lng': 34.809708, 'labeledL...","[{'id': '4bf58dd8d48988d145941735', 'name': 'C...",v-1583505798,False,NaN
2,4cc965dcbfe1f04dc7870d75,אצה סושי,"{'lat': 32.08323412282996, 'lng': 34.803849552...","[{'id': '4bf58dd8d48988d1d2941735', 'name': 'S...",v-1583505798,False,NaN
3,4ff17dfee4b018b0680ebaa9,צ׳אנג מאי,"{'lat': 32.08767781134958, 'lng': 34.803949225...","[{'id': '4bf58dd8d48988d149941735', 'name': 'T...",v-1583505799,False,NaN
4,5c41b198f870fd002c1b1deb,Ning,"{'lat': 32.067782, 'lng': 34.767961, 'labeledL...","[{'id': '4bf58dd8d48988d142941735', 'name': 'A...",v-1583505799,False,NaN


### Results

#### Clustering the restaurants by location:

As stated in the business problem at the start of my report, location is key.
For simplicity's sake, and since there are not that many Asian restaurants in Tel-Aviv, we are going to divide the city to 5 main boroughs (there may be some overlap, naturally, as this is a gross estimation):

1. Northern Tel Aviv (This uptown part of Tel-Aviv is very affluent, quiet, and is comprised of mostly upscale residential neighborhoods)
2. Central Tel Aviv (The bustling, chic part of the city where most things happen - most restaurants will probably be here)
3. Southern Tel Aviv (This part of Tel-Aviv is industrial and quite impoverished; it is very heterogenous due to many poor foreign immigrants having settled there, and is widely considered to be the "bad part of town")
4. Eastern Tel Aviv (Very Urban, middle-class, with some residential areas as well as commercial areas, mostly "run-of-the-mill" neighbourhoods, especially compared to Northern and Central Tel-Aviv)
5. Jaffa (Some could argue it is a distinct town onto itself, but as it is officialy part of Tel-Aviv, I included it too.)

###### It would be interesting to see how the restaurants are clustered by borough, and we will achieve that by stating some boundaries as constant coordinates, then adding circle markers to our Tel-Aviv map, to show the different boroughs' boundaries

In [7]:
TEL_AVIV_BOROUGHS = {
    'Northern Tel Aviv': {
        'center': (32.115974, 34.810723),
        'radius': 160,
        'color': 'blue'
    },
    'Central Tel Aviv': {
        'center': (32.081541, 34.781112),
        'radius': 150,
        'color': 'orange'
    },
    'Southern Tel Aviv': {
        'center': (32.055588, 34.777612),
        'radius': 115,
        'color': 'red'
    },
    'Eastern Tel Aviv': {
        'center': (32.055515, 34.804677),
        'radius': 100,
        'color': 'green'
    },
    'Jaffa': {
        'center': (32.051335, 34.755558),
        'radius': 110,
        'color': 'purple'
    }
}

for k,v in TEL_AVIV_BOROUGHS.items():
    folium.CircleMarker(
        location=v['center'],
        radius=v['radius'],
        popup=k,
        color=v['color'],
        fill=True,
        fill_color=v['color']
    ).add_to(telaviv_map)
    
telaviv_map

###### Nice, now we can show them on the map and see how they are spread:

In [8]:
for i in range(0, len(found_restaurants_df)):
    folium.Marker(
        [found_restaurants_df.iloc[i]['location']['lat'], found_restaurants_df.iloc[i]['location']['lng']],
        popup=found_restaurants_df.iloc[i]['name']
    ).add_to(telaviv_map)

telaviv_map

### Discussion

As can be seen from the above map, most Asian restaurants are located, perhaps quite expectedly so, in the heart of Central Tel-Aviv.
There is some overlap with the Southern districts, where we can also see some Asian restaurants.

What's interesting is that we barely see any Asian restaurants, if at all, in Jaffa, Eastern Tel-Aviv and Northern Tel-Aviv.
This begs the question: Would opening an Asian restaurant in one of these boroughs be a good investment, given the (very) minimal potential competition?

Let us consider each borough separately: 

- Jaffa: There are barely any Asian restaurants in most of Jaffa, with the only cluster being at the very edge of the borough, where it overlaps with both Southern and Central Tel-Aviv. This could be a great spot for an Asian restaurant, and in fact I'll admit it was my first thought - but seeing as how there are already many Asian restaurants within that area, I would shift my priorities somewhere else. The rest of Jaffa is empty for a reason - it is quite poor and undeveloped, and therefore a high-end Asian restaurant in that area would not make sense in the first place.

- Northern Tel-Aviv: This is a residential area and therefore the potential for customers is not very high. However, since my Asian restaurant is supposed to be high-end and aimed at affluent customers, I would probably prefer this borough to Jaffa. Another issue is that Northern Tel-Aviv is quite large and sparse, so my restaurant would not be as accessible.

- Eastern Tel-Aviv: All in all, especially having considered the two above boroughs, this borough makes most sense. We can see it has a nice overlap with both Southern and Central Tel-Aviv, however as oppposed to Jaffa and even Northern Tel-Aviv - there are absolutely no Asian restaurants in that area! This makes my Asian restaurant stand out, and it would be much more accessible given the said overlap. Ideally, I would open my Asian restaurant at the small intersection between the three areas (Where the green, red and orange balloons intersect).

### Conclusion

So, given the above discussion, it only makes sense that I open my high-end Asian restaurant at the intersection between Central, Eastern and Southern Tel-Aviv! It is easily accessible, it has good customer volume, there isn't much competition nearby and the Eastern & Central Tel-Aviv boroughs are affluent enough to attract high-paying customers.